# Non-linear Sigma Model with chemical potential

Sigma model Hamiltonian
$$ a\mathcal{H} = \frac{1}{2\beta}\sum_{k=1}^{N} \mathbf{J}_k^2 - \beta \sum_{k=1}^{N-1}n_k n_{k+1} - a\mu \mathbf{Q}$$
$n_k n_{k+1}$ can be rewritten as (see [this paper](https://link.aps.org/doi/10.1103/PhysRevD.99.074501))
$$ n_k n_{k+1} = n^+_k n^-_{k+1} + n^-_k n^+_{k+1} + n^z_k n^z_{k+1}$$


For 4D local Hilbert-space, these terms can be written as
$$ n^+ = \frac{1}{\sqrt{3}}\left(\begin{matrix} 0 & 0 & 0 & -1 \\ 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \end{matrix}\right)\quad , \quad
n^- = \frac{1}{\sqrt{3}}\left(\begin{matrix} 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ -1 & 0 & 0 & 0 \end{matrix}\right)\quad , \quad
n^z = \frac{1}{\sqrt{3}}\left(\begin{matrix} 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0 \\ 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \end{matrix}\right)\quad , \quad
Q = \left(\begin{matrix} 0 & 0 & 0 & 0 \\ 0 & -1 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 \end{matrix}\right)
$$
and the kinetic term
$$ \mathcal{H}^0 = \left(\begin{matrix} 0 & 0 & 0 & 0 \\ 0 & 2 & 0 & 0 \\ 0 & 0 & 2 & 0 \\ 0 & 0 & 0 & 2 \end{matrix}\right)$$

Note that these terms are originating from the truncated Hamiltonian, where normaly Hilbert space is infinite.

To use these on a quantum device, they need to be decomposed into Pauli basis i.e. $\{I, X, Y, Z\}$ which is equavalent to $\{ \mathbb{1}, \sigma_x, \sigma_y, \sigma_z\}$
$$ \mathcal{H}^0 = \frac{1}{2} (3\ II - IZ - ZI - ZZ )\quad , \quad \mathbf{Q} = \frac{1}{2}(ZZ - ZI)$$
$$ n_k n_{k+1} = \frac{1}{24}(IXIX-IXXX+IXYY+IXZX+IYIY+IYXY+IYYX+IYZY+2XIXI+2XIXZ-XXIX+XXXX-XXYY-XXZX+XYIY+XYXY+XYYX+XYZY+2XZXI+2XZXZ+YXIY+YXXY+YXYX+YXZY+YYIX-YYXX+YYYY+YYZX+ZXIX-ZXXX+   ZXYY+ZXZX+ZYIY+ZYXY+ZYYX+ZYZY)
$$


**Notation example:** $\sigma_x^0\otimes\sigma_y^1 = XY$

In [1]:
import pennylane as qml
import pennylane.numpy as np
import matplotlib.pyplot as plt

In [2]:
beta = np.linspace(0.7, 1.8, 12)
mu   = np.linspace(0.3, 0.7, 9)

## Construct Hamiltonian using Pennylane

Notice that the Hamiltonian below is written for 3 sites with periodic boundary conditions, so the graph structure is triangular.
Lets first try to obtain result for this construction and then move on for larger number of sites and different geometries. Observe how 
does the result change with lattice size. 

In [ ]:
def Hamiltonian(beta: float, mu: float = 0.):
    div_2beta = 1./(2*beta)
    # last two terms include Q in the kinetic term
    kinetic = lambda n0, n1 : [qml.Identity(n0) @ qml.Identity(n1), 
                               qml.Identity(n0) @ qml.PauliZ(n1),
                               qml.PauliZ(n0) @ qml.Identity(n1),
                               qml.PauliZ(n0) @ qml.PauliZ(n1),
                               qml.PauliZ(n0) @ qml.Identity(n1),
                               qml.PauliZ(n0) @ qml.PauliZ(n1)]

    kinetic_term = kinetic(0,1) + kinetic(2,3) + kinetic(4,5)
    kinetic_coeff = [1.5 * div_2beta, -0.5 * div_2beta, -0.5 * div_2beta, -0.5 * div_2beta, 0.5 * mu, -0.5 * mu] * 3
    
    interaction = lambda n0, n1, n2, n3: [qml.Identity(n0) @ qml.PauliX(n1) @ qml.Identity(n2) @ qml.PauliX(n3),
                                          qml.Identity(n0) @ qml.PauliX(n1) @ qml.PauliX(n2) @ qml.PauliX(n3),
                                          qml.Identity(n0) @ qml.PauliX(n1) @ qml.PauliY(n2) @ qml.PauliY(n3),
                                          qml.Identity(n0) @ qml.PauliX(n1) @ qml.PauliZ(n2) @ qml.PauliX(n3),
                                          qml.Identity(n0) @ qml.PauliY(n1) @ qml.Identity(n2) @ qml.PauliY(n3),
                                          qml.Identity(n0) @ qml.PauliY(n1) @ qml.PauliX(n2) @ qml.PauliY(n3),
                                          qml.Identity(n0) @ qml.PauliY(n1) @ qml.PauliY(n2) @ qml.PauliX(n3),
                                          qml.Identity(n0) @ qml.PauliY(n1) @ qml.PauliZ(n2) @ qml.PauliY(n3),
                                          qml.PauliX(n0) @ qml.Identity(n1) @ qml.PauliX(n2) @ qml.Identity(n3),
                                          qml.PauliX(n0) @ qml.Identity(n1) @ qml.PauliX(n2) @ qml.PauliZ(n3),
                                          qml.PauliX(n0) @ qml.PauliX(n1) @ qml.Identity(n2) @ qml.PauliX(n3),
                                          qml.PauliX(n0) @ qml.PauliX(n1) @ qml.PauliX(n2) @ qml.PauliX(n3),
                                          qml.PauliX(n0) @ qml.PauliX(n1) @ qml.PauliY(n2) @ qml.PauliY(n3),
                                          qml.PauliX(n0) @ qml.PauliX(n1) @ qml.PauliZ(n2) @ qml.PauliX(n3),
                                          qml.PauliX(n0) @ qml.PauliY(n1) @ qml.Identity(n2) @ qml.PauliY(n3),
                                          qml.PauliX(n0) @ qml.PauliY(n1) @ qml.PauliX(n2) @ qml.PauliY(n3),
                                          qml.PauliX(n0) @ qml.PauliY(n1) @ qml.PauliY(n2) @ qml.PauliX(n3),
                                          qml.PauliX(n0) @ qml.PauliY(n1) @ qml.PauliZ(n2) @ qml.PauliY(n3),
                                          qml.PauliX(n0) @ qml.PauliZ(n1) @ qml.PauliX(n2) @ qml.Identity(n3),
                                          qml.PauliX(n0) @ qml.PauliZ(n1) @ qml.PauliX(n2) @ qml.PauliZ(n3),
                                          qml.PauliY(n0) @ qml.PauliX(n1) @ qml.Identity(n2) @ qml.PauliY(n3),
                                          qml.PauliY(n0) @ qml.PauliX(n1) @ qml.PauliX(n2) @ qml.PauliY(n3),
                                          qml.PauliY(n0) @ qml.PauliX(n1) @ qml.PauliY(n2) @ qml.PauliX(n3),
                                          qml.PauliY(n0) @ qml.PauliX(n1) @ qml.PauliZ(n2) @ qml.PauliY(n3),
                                          qml.PauliY(n0) @ qml.PauliY(n1) @ qml.Identity(n2) @ qml.PauliX(n3),
                                          qml.PauliY(n0) @ qml.PauliY(n1) @ qml.PauliX(n2) @ qml.PauliX(n3),
                                          qml.PauliY(n0) @ qml.PauliY(n1) @ qml.PauliY(n2) @ qml.PauliY(n3),
                                          qml.PauliY(n0) @ qml.PauliY(n1) @ qml.PauliZ(n2) @ qml.PauliX(n3),
                                          qml.PauliZ(n0) @ qml.PauliX(n1) @ qml.Identity(n2) @ qml.PauliX(n3),
                                          qml.PauliZ(n0) @ qml.PauliX(n1) @ qml.PauliX(n2) @ qml.PauliX(n3),
                                          qml.PauliZ(n0) @ qml.PauliX(n1) @ qml.PauliY(n2) @ qml.PauliY(n3),
                                          qml.PauliZ(n0) @ qml.PauliX(n1) @ qml.PauliZ(n2) @ qml.PauliX(n3),
                                          qml.PauliZ(n0) @ qml.PauliY(n1) @ qml.Identity(n2) @ qml.PauliY(n3),
                                          qml.PauliZ(n0) @ qml.PauliY(n1) @ qml.PauliX(n2) @ qml.PauliY(n3),
                                          qml.PauliZ(n0) @ qml.PauliY(n1) @ qml.PauliY(n2) @ qml.PauliX(n3),
                                          qml.PauliZ(n0) @ qml.PauliY(n1) @ qml.PauliZ(n2) @ qml.PauliY(n3)]
    interaction_term = interaction(0,1,2,3) + interaction(2,3,4,5) + interaction(4,5,0,1)
    
    interaction_coeff = [1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0,
                         1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0,
                         1.0, 1.0, 1.0, 1.0] * 3
    interaction_coeff = [-beta * x / 24. for x in interaction_coeff]
    
    return qml.Hamiltonian(kinetic_coeff + interaction_coeff, kinetic_term + interaction_term)

Useful functions:
* ``qml.state()`` returns the state of the quantum circuit
* ``qml.expval()`` computes expectation value with respect to an operator

**First step:** Try to compute phase transition using exact diagonalisation.

**Second step:** Find a suitable ansatz that can diagonalise the Hamiltonian. Do you need to change the ansatz for large $\beta$ or $\mu$?

**Third step:** Compute the lowest eigenstate for $\mu\in[0.3,0.7]$ and $\beta\in[0.7,1.8]$

**Fourth step:** plot $\langle Q\rangle$

* Do you see any phase transition? 
* How does it compare to the results that you obtained with exactdiagonalisation?
* How can you improve these results by changing your ansatz and/or your training procedure?
* Have you tried different traning algorithms: ADAM, SGD, QNGD etc?
* How does it change with larger lattice sizes and different geometries?

### Further reading:
* [O(3) nonlinear sigma model in 1+1 dimensions with matrix product states](https://arxiv.org/abs/1812.00944)
* [Toward a quantum simulation of nonlinear sigma models with a topological term](https://arxiv.org/abs/2210.03679)